In [1]:
import pandas as pd 
import numpy as np 


In [17]:
# df = pd.read_excel('Refund Application/2_Sales Report.xlsx',header=None,sheet_name='B2B')
# start_index = 0
# for row in df.itertuples():
#     if  'Return Period' in row:
#         start_index =row.Index
# frame = df.loc[start_index+1:]
# column_names = df.loc[start_index]
# frame.columns = column_names


In [33]:
def create_df(sheet_name):
    df = pd.read_excel('Refund Application/2_Sales Report.xlsx',header=None,sheet_name=sheet_name)
    start_index = 0
    for row in df.itertuples():
        if  'Return Period' in row:
            start_index =row.Index
    frame = df.loc[start_index+1:]
    column_names = df.loc[start_index]
    frame.columns = column_names
    return frame

In [84]:
def get_sales():
    file_path = 'Refund Application/2_Sales Report.xlsx'
    purchase = pd.ExcelFile(file_path)
    sheet_dataframes = []
    for sheet in purchase.sheet_names:
        
        df = create_df(sheet_name=sheet)
        if sheet == 'B2CS':
            df['Type of Outward Supply'] = 'B2C-Small'
            df['Type of Document'] = 'Invoice/Bill of Entry'
        elif sheet == 'CDNR' :
            df['Type of Outward Supply'] = 'B2B'
            df['Type of Document'] = 'Credit Note'
        elif sheet == 'B2B':
            df['Type of Outward Supply'] = 'B2B'
            df['Type of Document'] = 'Invoice/Bill of Entry'



        if sheet != 'EXP':
            print(sheet)
            sheet_dataframes.append(df)

    return sheet_dataframes


sales_dataframes = get_sales()


B2B
B2CS
CDNR


In [85]:
sales_dataframes[1].head()

7,Return Period,Original Period,POS,Taxable Value,Tax Rate,IGST,CGST,SGST,Cess,Type of Outward Supply,Type of Document
8,"February,2024",NaN,34-Puducherry,9519.22,5,475.96,0,0,0,B2C-Small,Invoice/Bill of Entry
9,"January,2024",NaN,27-Maharashtra,9428.56,5,0,235.71,235.71,0,B2C-Small,Invoice/Bill of Entry
10,"January,2024",NaN,29-Karnataka,46142.92,5,2307.15,0,0,0,B2C-Small,Invoice/Bill of Entry
11,"January,2024",NaN,33-Tamil Nadu,14344.72,5,717.24,0,0,0,B2C-Small,Invoice/Bill of Entry
12,"March,2024",NaN,27-Maharashtra,42718.9,5,0,1067.97,1067.97,0,B2C-Small,Invoice/Bill of Entry


In [86]:
def condense_dataframe(df):
    condensed_data = {}

    for col in df.columns:
        try:
            # Try converting column to numeric (force errors to NaN), then sum
            numeric_col = pd.to_numeric(df[col], errors='coerce')
            if numeric_col.notna().all():  # if all values are numeric
                condensed_data[col] = numeric_col.sum()
            else:
                raise ValueError  # treat as non-numeric
        except:
            # If conversion fails or not fully numeric, keep first value
            condensed_data[col] = df[col].iloc[0]

    return pd.DataFrame([condensed_data])

In [87]:
## we have the sales data frames with us

sales_final_columns = ['Type of Outward Supply' , 'Type of Document','No','Date','Taxable Value','Integrated Tax','Central Tax','State/Union Territory Tax' ]

In [88]:
cols_mapping = {
    'Type of Outward Supply':'Type of Outward Supply',
    'Type of Document':'Type of Document',
    'No':'Invoice No',
    'Date': "Invoice Date",
    'Taxable Value': 'Taxable Value',
    'Integrated Tax': 'IGST',
    'Central Tax': 'CGST',
    'State/Union Territory Tax':'SGST',
}
cols_mapping_cdnr = {
    'Type of Outward Supply':'Type of Outward Supply',
    'Type of Document':'Type of Document',
    'No':'Note No',
    'Date': "Note Date",
    'Taxable Value': 'Taxable Value',
    'Integrated Tax': 'IGST',
    'Central Tax': 'CGST',
    'State/Union Territory Tax':'SGST',
}
# invoice col prob
# note date

In [110]:
def create_final_sales_frame(dfs, cols_mapping,cols_mapping_cdnr):
    new_dfs = []
    for df in dfs:
        # if else based on type of doc , for cdnr different mapping
        if 'Credit Note' in  df['Type of Document'].value_counts().keys() :
            # Filter only columns present in the current df
            available_mapping = {k: v for k, v in cols_mapping_cdnr.items() if v in df.columns}
            temp_df = df[list(available_mapping.values())].rename(columns={v: k for k, v in available_mapping.items()})
            temp_df['Date'] = pd.to_datetime(temp_df['Date']).dt.strftime('%d-%m-%Y')
        else:
            available_mapping = {k: v for k, v in cols_mapping.items() if v in df.columns}
            temp_df = df[list(available_mapping.values())].rename(columns={v: k for k, v in available_mapping.items()})
            if "Date" not in temp_df.columns:
                temp_df["Date"] = ''
            
            if 'B2C-Small' in temp_df['Type of Outward Supply'].value_counts().keys() :
                temp_df['No'] = 'B2COTH'
                temp_df['Date'] = ''
                # print(temp_df.info())
                temp_df = condense_dataframe(temp_df)
                
            # now here we have b2cs and b2b , so we ll check if it is b2cs we ll sum the data and add
        temp_df['Date'] = pd.to_datetime(temp_df['Date']).dt.strftime('%d-%m-%Y')
        print(temp_df.shape)
        new_dfs.append(temp_df)

    return pd.concat(new_dfs, ignore_index=True)

# Call the function
final_df = create_final_sales_frame(sales_dataframes, cols_mapping,cols_mapping_cdnr)

(255, 8)
(1, 8)
(6, 8)


/var/folders/vv/bzfb0zp975s_s9tp9_ydn6sw0000gn/T/ipykernel_86540/3572685943.py:23: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  temp_df['Date'] = pd.to_datetime(temp_df['Date']).dt.strftime('%d-%m-%Y')


In [111]:
final_df.head()
# if 'Inward Supply from Registered Person' in df['Type of Inward Supply'].value_counts().keys():

,Type of Outward Supply,Type of Document,No,Date,Taxable Value,Integrated Tax,Central Tax,State/Union Territory Tax
0,B2B,Invoice/Bill of Entry,2023-24/771,01-01-2024,34194.32,1709.76,0,0
1,B2B,Invoice/Bill of Entry,2023-24/772,01-01-2024,71011.48,3550.64,0,0
2,B2B,Invoice/Bill of Entry,2023-24/773,01-01-2024,25084.18,1254.22,0,0
3,B2B,Invoice/Bill of Entry,2023-24/783,02-01-2024,70948.44,3547.56,0,0
4,B2B,Invoice/Bill of Entry,2023-24/776,02-01-2024,88350.46,0,2208.77,2208.77


In [112]:
final_df.to_excel('sales1.xlsx')